In [1]:
import numpy as np
from numba import njit, vectorize, guvectorize, prange

In [ ]:
@njit
def scalar_vector_jit(x, y, res):
    for i in range(y.size):
        res[i] = x*y[i]
    return res

In [ ]:
@njit(parallel=True)
def scalar_vector_jit_par(x, y, res):
    for i in prange(y.size):
        res[i] = x*y[i]
    return res

In [ ]:
@guvectorize('float64, float64[:], float64[:]', '(),(n)->(n)')
def scalar_vector_guv(x, y, res):
    for i in range(y.size):
        res[i] = x*y[i]

In [ ]:
@guvectorize('float64, float64[:], float64[:]', '(),(n)->(n)',
             target='parallel')
def scalar_vector_guv_par(x, y, res):
    for i in range(y.size):
        res[i] = x*y[i]

In [ ]:
y = np.ones(10000)

In [ ]:
result = np.zeros_like(y)

In [ ]:
%timeit 3.*y

In [ ]:
%timeit scalar_vector_jit(3., y, result)

In [ ]:
%timeit scalar_vector_jit_par(3., y, result)

In [ ]:
%timeit scalar_vector_guv(3., y, result)

In [ ]:
%timeit scalar_vector_guv_par(3., y, result)

In [2]:
@njit(parallel=True)
def outer_jit(x, y):
    res = np.empty((x.size,y.size))
    for i in prange(x.size):
        for j in range(y.size):
            res[i,j] = x[i]*y[j]
    return res

In [3]:
@guvectorize('float64[:], float64[:], float64[:,:]', '(m),(n)->(m,n)')
def outer_guv(x, y, res):
    for i in range(x.size):
        for j in range(y.size):
            res[i,j] = x[i]*y[j]

In [10]:
@guvectorize('float64[:], float64[:], float64[:,:]', '(m),(n)->(m,n)',
             target='parallel')
def outer_guv_par(x, y, res):
    for i in range(x.size):
        for j in range(y.size):
            res[i,j] = x[i]*y[j]

In [5]:
y = np.ones(3000)

In [6]:
res = np.empty_like(y)

In [7]:
%timeit np.outer(y, y)

17.7 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit outer_jit(y, y)

8.9 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit outer_guv(y, y, res)

ValueError: outer_guv: Output operand 0 does not have enough dimensions (has 1, gufunc core with signature (m),(n)->(m,n) requires 2)

In [11]:
%timeit outer_guv_par(y, y, res)

ValueError: outer_guv_par: Output operand 0 does not have enough dimensions (has 1, gufunc core with signature (m),(n)->(m,n) requires 2)